In [168]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import ib.opt
import ib.ext.Contract
from ib.ext.TickType import TickType as tt
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [129]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 5

        self.con = ib.opt.ibConnection(**kwargs)
        #self.con.registerAll(self.watcher)
        
        self.contracts = pd.DataFrame([
                ['IBM','SMART','USD'],
                ['AAPL','SMART','USD'],
                ['GOOG','SMART','USD'],
                ['ES','GLOBEX','USD','201506','50'],
                ['CL','NYMEX','USD','201506','1000']
        ],columns = ['sym','exch','curr','expiry','mult'])

        #add these specific column names to match the name returned by TickType.getField()
        self.contracts['bidPrice'] = 0
        self.contracts['askPrice'] = 0
        self.contracts['lastPrice'] = 0


        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler,
            ib.opt.message.tickPrice: self.tickPriceHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()
        
    def tickPriceHandler(msg):
        print (msg)
        if msg.field in [tt.BID,tt.ASK,tt.LAST]:
            #now we can just store the response in the data frame
            self.contracts.loc[msg.tickerId,tt.getField(msg.field)] = msg.price
            if msg.field == tt.LAST:
                print(contracts.loc[msg.tickerId,'sym'],msg.price)

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))

    def get_contract_details(self, contract):
        self.con.reqContractDetails(1, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            if df.shape[0]==0:
                df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            else:
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)

        return df, err, mid, msg

In [130]:

def esFutContract():
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    
    return contract

def esOptContract(exp):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
 
    return contract

In [157]:
ibm.con.disconnect()
#ibm = IbManager("127.0.0.1", port=7497, clientId=999)

True

In [132]:
def vxOptContract(exp):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='VIX'
 
    return contract

def spOptContract(exp):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='SP'
 
    return contract

def vixFutContract(localSym=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'
    contract.m_localSymbol = localSym

    return contract

def spFutContract(exp=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'SP'
    contract.m_expiry = exp
    
    return contract

In [120]:
#cdetails, err, errid, errmsg = ibm.get_contract_details(vixFutContract('VXZ8'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(esFutContract())
#cdetails, err, errid, errmsg = ibm.get_contract_details(esOptContract('20190315'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(vxOptContract('20190115'))
cdetails, err, errid, errmsg = ibm.get_contract_details(spOptContract('20190118'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(spFutContract('20190314'))

In [141]:
for index, row in contracts.iterrows():
    c = Contract()
    c.m_symbol = row['sym']
    c.m_exchange = row['exch']
    c.m_currency = row['curr']
    c.m_secType = 'STK' if row['expiry'] is None else 'FUT'
    c.m_expiry = row['expiry']
    c.m_multiplier = row['mult']
    # the tickerId is just the index in but for some reason it needs str()
    
    break

In [153]:
def savepx(msg):
    print(msg.price)

In [164]:
con.disconnect()
con=ib.opt.ibConnection("127.0.0.1", port=7497, clientId=999)
con.register(savepx, ib.opt.message.tickPrice)
con.connect()

Server Version: 76
TWS Time at connection:20181218 13:04:56 EST


True

In [165]:
con.reqMktData(1101, c, "", False)

In [162]:
c = Contract()
c.m_symbol = 'AAPL'
con.register(savepx,'TickPrice')# move this out of loop, just do it once

In [170]:
con.reqMktData(1000, c, "", False)